In [44]:
#!pip install plac
#!pip install -U spacy
!python -m spacy download en_core_web_md
!pip install nervaluate

^C
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py38/lib/python3.8/runpy.py", line 185, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/anaconda/envs/azureml_py38/lib/python3.8/runpy.py", line 144, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/anaconda/envs/azureml_py38/lib/python3.8/runpy.py", line 111, in _get_module_details
    __import__(pkg_name)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/spacy/__init__.py", line 14, in <module>
    from . import pipeline  # noqa: F401
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/spacy/pipeline/__init__.py", line 1, in <module>
    from .attributeruler import AttributeRuler
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/spacy/pipeline/attributeruler.py", line 6, in <module>
    from .pipe import Pipe
  File "spacy/pipeline/pipe.pyx", line 8, in init spacy.pipeline.pipe
  File "/an

In [1]:
from __future__ import unicode_literals, print_function
import pickle
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
from spacy.training import Example
import srsly
import pickle
import os
import csv
import numpy as np
import time
import json
from nervaluate import Evaluator

In [2]:
spacy.__version__

'3.1.3'

In [21]:
output_dir = "data"
result_dir = "result"

In [14]:
LABEL = ["base", "aspect_changing", "change_direction", "type_of"]

with open (output_dir + "/training_data.txt", 'rb') as file:
    TRAIN_DATA = pickle.load(file)

with open (output_dir + "/testing_data.txt", 'rb') as file:
    TEST_DATA = pickle.load(file)

In [15]:
model = "en_core_web_md"
new_model_name = "custom_entity_extraction"
n_iter = 200
batch_size = 50

In [16]:
print(len(TRAIN_DATA))
print(len(TEST_DATA))

449
50


In [25]:
TEST_DATA

[('His research team compared known health impacts from air pollution against future climate scenarios to derive its projections. IMPLIED_BASE IMPLIED_BASE',
  {'entities': [(33, 39, 'base'),
    (40, 47, 'aspect_changing'),
    (53, 56, 'type_of'),
    (57, 66, 'base')]}),
 ('Among its impacts, the heatwave caused crop failures across Europe, fanned wildfires from Manchester in the UK to Yosemite National Park in the US and exposed more than 34,000 people to power outages in Los Angeles as the grid experienced an unprecedented demand for air conditioning. IMPLIED_BASE IMPLIED_BASE',
  {'entities': [(23, 31, 'base'),
    (39, 43, 'base'),
    (44, 52, 'aspect_changing'),
    (68, 74, 'change_direction'),
    (75, 84, 'base'),
    (186, 191, 'base'),
    (192, 199, 'aspect_changing'),
    (222, 226, 'base'),
    (256, 262, 'aspect_changing'),
    (267, 283, 'base')]}),
 ('Regenerative farming, rotational and mob grazing, reducing tillage, agroforestry, and soil improvements are all mean

In [20]:
print("starting time: " + time.strftime('%m/%d/%Y %H:%M:%S', time.localtime(time.time())))
if model is not None:
    nlp = spacy.load(model)
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')
    print("Created blank 'en' model")
    
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe('ner')

for i in LABEL:
    # Adding new entity
    ner.add_label(i)

if model is None:
    optimizer = nlp.begin_training()
else:
    #optimizer = nlp.entity.create_optimizer()
    optimizer = nlp.create_optimizer()
    
# Get names of other pipes to disable them during training to train only NER
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
        #for batch in batches:
        #    texts, annotations = zip(*batch)
        #    nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)
        #print('Losses', losses)
        for batch in spacy.util.minibatch(TRAIN_DATA, size=batch_size):
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)
        print(str(itn) + " loss: ", losses)
        
# Save model 
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.meta['name'] = new_model_name  # rename model
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

print("ending time: " + time.strftime('%m/%d/%Y %H:%M:%S', time.localtime(time.time())))

starting time: 09/28/2022 18:48:32
Loaded model 'en_core_web_md'
0 loss:  {'ner': 4627.446830126743}
1 loss:  {'ner': 3614.1820618331544}
2 loss:  {'ner': 3152.8385711346823}
3 loss:  {'ner': 2836.829549912623}
4 loss:  {'ner': 2551.450287978364}
5 loss:  {'ner': 2360.723427600743}
6 loss:  {'ner': 2183.280440711687}
7 loss:  {'ner': 2103.070454425247}
8 loss:  {'ner': 1955.910360846576}
9 loss:  {'ner': 1968.3071469821134}
10 loss:  {'ner': 1816.3685661763102}
11 loss:  {'ner': 1744.216094576551}
12 loss:  {'ner': 1637.8667575899863}
13 loss:  {'ner': 1564.1229956903442}
14 loss:  {'ner': 1471.4620383356441}
15 loss:  {'ner': 1367.3956451999088}
16 loss:  {'ner': 1377.172620356496}
17 loss:  {'ner': 1297.544757553306}
18 loss:  {'ner': 1284.4496535209314}
19 loss:  {'ner': 1204.3186212644034}
20 loss:  {'ner': 1138.6926340063276}
21 loss:  {'ner': 1235.3325205859392}
22 loss:  {'ner': 1089.465987110618}
23 loss:  {'ner': 1131.4532132360669}
24 loss:  {'ner': 1071.0334421993246}
25 los

In [22]:
# Test the saved model
print("Loading from", output_dir)
nlp_test = spacy.load(output_dir)

Loading from data


In [40]:
result = os.path.join(output_dir, "spacy_predictions.txt")

final_res = []
PRED_DATA = []
for sent in TEST_DATA:
    doc2 = nlp_test(sent[0])
    PRED_DATA.append(doc2.to_json()["ents"])


In [42]:
PRED_DATA

[[{'start': 33, 'end': 39, 'label': 'base'},
  {'start': 40, 'end': 47, 'label': 'aspect_changing'},
  {'start': 53, 'end': 56, 'label': 'type_of'},
  {'start': 57, 'end': 66, 'label': 'base'},
  {'start': 67, 'end': 126, 'label': 'type_of'},
  {'start': 127, 'end': 139, 'label': 'base'}],
 [{'start': 23, 'end': 31, 'label': 'base'},
  {'start': 39, 'end': 43, 'label': 'type_of'},
  {'start': 44, 'end': 52, 'label': 'base'},
  {'start': 68, 'end': 74, 'label': 'change_direction'},
  {'start': 75, 'end': 84, 'label': 'base'},
  {'start': 151, 'end': 158, 'label': 'type_of'},
  {'start': 159, 'end': 175, 'label': 'change_direction'},
  {'start': 176, 'end': 182, 'label': 'base'},
  {'start': 186, 'end': 199, 'label': 'base'},
  {'start': 256, 'end': 262, 'label': 'type_of'},
  {'start': 267, 'end': 270, 'label': 'type_of'},
  {'start': 271, 'end': 283, 'label': 'base'}],
 [{'start': 0, 'end': 12, 'label': 'type_of'},
  {'start': 13, 'end': 20, 'label': 'base'},
  {'start': 37, 'end': 40,

In [48]:
def create_nerevaluate_file(data):
    ans = []
    for s in data:
        tmp = []
        for e in s[1]["entities"]:
            ans_dict = {}
            ans_dict["start"] = e[0]
            ans_dict["end"] = e[1]
            ans_dict["label"] = e[2]
            tmp.append(ans_dict)
        ans.append(tmp)

    return ans

actual_answers = create_nerevaluate_file(TEST_DATA)

assert len(PRED_DATA) == len(actual_answers)

evaluator = Evaluator(actual_answers, PRED_DATA, tags=LABEL)

results, results_per_tag = evaluator.evaluate()
#print(results)
#print()
#print(results_per_tag)

with open(result_dir + "/results_SPACY.json", "w") as f:
    json.dump(results, f, indent=4)
with open(result_dir + "/results_per_tag_SPACY.json", "w") as f:
    json.dump(results_per_tag, f, indent=4)